In [1]:
import os
import csv
from towhee import ops, pipe, register
from towhee.operator import PyOperator
from towhee import DataCollection
from tqdm import tqdm
import pandas as pd
import json
import numpy as np
from helpers import milvus_utils

Connected to Milvus server at port 19530


In [ ]:
# CONSTANTS

# Files
MSRVTT_SAMPLES = "./MSRVTT_1K.csv"
# file created using raw FIRE judgements, see clean_fire_judgements.ipynb
FIRE_BENCHMARK_Q_JUDGEMENTS = "./fire_benchmark_q_judgements.csv" 

# Database Collections
VIDEO_RET_COLLECTION = "msrvtt_vid_ret_1"
FRAME_RET_COLLECTION = "msrvtt_frame_ret_1"


In [4]:

raw_samples_df = pd.read_csv(MSRVTT_SAMPLES)
raw_samples_df[['video_id', 'video_path', 'sentence']].head()

,video_id,video_path,sentence
0,video7579,./test_1k_compress/video7579.mp4,a girl wearing red top and black trouser is pu...
1,video7725,./test_1k_compress/video7725.mp4,young people sit around the edges of a room cl...
2,video9258,./test_1k_compress/video9258.mp4,a person is using a phone
3,video7365,./test_1k_compress/video7365.mp4,cartoon people are eating at a restaurant
4,video8068,./test_1k_compress/video8068.mp4,a woman on a couch talks to a a man


In [ ]:
milvus_utils.create_milvus_collection(VIDEO_RET_COLLECTION, 512)

<Collection>:
-------------
<name>: msrvtt_vid_ret_1
<description>: video retrieval
<schema>: {'auto_id': False, 'description': 'video retrieval', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 512}}]}

We create a pipeline that loads the video embeddings into the Milvus Vector DB using a distributed Twohee pipeline

In [ ]:
def read_loader_csv(csv_file):
    with open(csv_file, 'r', encoding='utf-8-sig') as f:
        data = csv.DictReader(f)
        for line in data:
            yield int(line['video_id'][len('video'):]), line['video_path']

video_loader_pipeline = (
    pipe.input('csv_file')
    .flat_map('csv_file', ('video_id', 'video_path'), read_loader_csv)
    # Create 12 evenly distributed frames per video
    .map('video_path', 'frames', ops.video_decode.ffmpeg(sample_type='uniform_temporal_subsample', args={'num_samples': 12}))
    # I have a M2 Max, so device is set to mps for better performance
    .map('frames', 'vec', ops.video_text_embedding.clip4clip(model_name='clip_vit_b32', modality='video', device='mps'))
    .map(('video_id', 'vec'), (), ops.ann_insert.milvus_client(collection_name=VIDEO_RET_COLLECTION))
    .output('video_id')
)

In [11]:
# We call the pipeline with a CSV file containing the video paths
video_loader_ret = video_loader_pipeline(MSRVTT_SAMPLES)

2025-04-16 15:04:15,510 - 17173606400 - node.py-node:167 - INFO: Begin to run Node-_input
2025-04-16 15:04:15,510 - 17190432768 - node.py-node:167 - INFO: Begin to run Node-read_loader_csv-0
2025-04-16 15:04:15,511 - 17207259136 - node.py-node:167 - INFO: Begin to run Node-video-decode/ffmpeg-1
2025-04-16 15:04:15,511 - 17224085504 - node.py-node:167 - INFO: Begin to run Node-video-text-embedding/clip4clip-2
2025-04-16 15:04:15,512 - 17240911872 - node.py-node:167 - INFO: Begin to run Node-ann-insert/milvus-client-3
2025-04-16 15:04:15,512 - 17173606400 - node.py-node:167 - INFO: Begin to run Node-_output


The 1000 videos are now loaded into the Milvus `VIDEO_RET_COLLECTION` collection.

Now, we query these videos using the annotated sentences as queries and the video ids as the ground truth results.

In [ ]:
def read_video_search_csv(csv_file):
    with open(csv_file, 'r', encoding='utf-8-sig') as f:
        data = csv.DictReader(f)
        for line in data:
            yield line['video_id'], line['sentence']

video_search_pipeline = (
    pipe.input('csv_file')
    .flat_map('csv_file', ('rel_video_id', 'query'), read_video_search_csv)
    .map('query', 'vec', ops.video_text_embedding.clip4clip(model_name='clip_vit_b32', modality='text', device='mps'))
    .map('vec', 'top10_raw_res', 
         ops.ann_search.milvus_client(collection_name=VIDEO_RET_COLLECTION, limit=10))
    .map('top10_raw_res', ('top1', 'top5', 'top10'), lambda x: (x[:1], x[:5], x[:10]))
    .output('rel_video_id', 'query', 'top1', 'top5', 'top10')
)

In [ ]:
all_query_results = DataCollection(video_search_pipeline(MSRVTT_SAMPLES))

2025-04-16 15:12:12,149 - 17800654848 - node.py-node:167 - INFO: Begin to run Node-_input
2025-04-16 15:12:12,150 - 17842597888 - node.py-node:167 - INFO: Begin to run Node-read_video_search_csv-0
2025-04-16 15:12:12,150 - 17976815616 - node.py-node:167 - INFO: Begin to run Node-video-text-embedding/clip4clip-1
2025-04-16 15:12:12,150 - 19253981184 - node.py-node:167 - INFO: Begin to run Node-ann-search/milvus-client-2
2025-04-16 15:12:12,150 - 19273904128 - node.py-node:167 - INFO: Begin to run Node-lambda-3
2025-04-16 15:12:12,151 - 21981327360 - node.py-node:167 - INFO: Begin to run Node-_output


2025-04-16 16:26:41,865 - 17800654848 - node.py-node:167 - INFO: Begin to run Node-_input
2025-04-16 16:26:41,865 - 17842597888 - node.py-node:167 - INFO: Begin to run Node-read_video_search_csv-0
2025-04-16 16:26:41,865 - 17976815616 - node.py-node:167 - INFO: Begin to run Node-video-text-embedding/clip4clip-1
2025-04-16 16:26:41,865 - 19253981184 - node.py-node:167 - INFO: Begin to run Node-ann-search/milvus-client-2
2025-04-16 16:26:41,865 - 19273904128 - node.py-node:167 - INFO: Begin to run Node-lambda-3
2025-04-16 16:26:41,865 - 21981327360 - node.py-node:167 - INFO: Begin to run Node-_output
2025-04-16 16:26:41,869 - 17842597888 - node.py-node:142 - INFO: read_video_search_csv-0 ends with status: NodeStatus.FAILED
2025-04-16 16:27:03,120 - 17800654848 - node.py-node:167 - INFO: Begin to run Node-_input
2025-04-16 16:27:03,120 - 17842597888 - node.py-node:167 - INFO: Begin to run Node-read_video_search_csv-0
2025-04-16 16:27:03,120 - 17976815616 - node.py-node:167 - INFO: Begin t

In [31]:
all_query_results.show()

rel_video_id,query,top1,top5,top10
video7579,a girl wearing red top and black trouser is putting a sweater on a dog,"[[7579, 1.415151834487915]] len=1","[[7579, 1.415151834487915],[9969, 1.4799106121063232],[8837, 1.4897732734680176],[9347, 1.4948583841323853],...] len=5","[[7579, 1.415151834487915],[9969, 1.4799106121063232],[8837, 1.4897732734680176],[9347, 1.4948583841323853],...] len=10"
video7725,young people sit around the edges of a room clapping and raising their arms while others dance in the center during a party,"[[7725, 1.3622068166732788]] len=1","[[7725, 1.3622068166732788],[8014, 1.4865269660949707],[8339, 1.4922078847885132],[8442, 1.5024112462997437],...] len=5","[[7725, 1.3622068166732788],[8014, 1.4865269660949707],[8339, 1.4922078847885132],[8442, 1.5024112462997437],...] len=10"
video9258,a person is using a phone,"[[9258, 1.4011969566345215]] len=1","[[9258, 1.4011969566345215],[9257, 1.422863483428955],[9697, 1.4413853883743286],[7910, 1.4945621490478516],...] len=5","[[9258, 1.4011969566345215],[9257, 1.422863483428955],[9697, 1.4413853883743286],[7910, 1.4945621490478516],...] len=10"
video7365,cartoon people are eating at a restaurant,"[[7365, 1.4027695655822754]] len=1","[[7365, 1.4027695655822754],[8781, 1.4623048305511475],[9537, 1.4739768505096436],[7831, 1.505112648010254],...] len=5","[[7365, 1.4027695655822754],[8781, 1.4623048305511475],[9537, 1.4739768505096436],[7831, 1.505112648010254],...] len=10"
video8068,a woman on a couch talks to a a man,"[[7162, 1.4716743230819702]] len=1","[[7162, 1.4716743230819702],[8304, 1.4787472486495972],[8068, 1.4926888942718506],[7724, 1.4982554912567139],...] len=5","[[7162, 1.4716743230819702],[8304, 1.4787472486495972],[8068, 1.4926888942718506],[7724, 1.4982554912567139],...] len=10"


In [ ]:
# Convert the Twohee data collection to a pandas dataframe so we can apply evaluation methods

In [ ]:
from helpers import eval_utils
# TODO rerun these 

In [ ]:
all_query_results_df = twohee_data_col_to_df(all_query_results)
# Create ground truth column as video id casted into an int, this is used for the eval functions
all_query_results_df['ground_truth'] = all_query_results_df['rel_video_id'].apply(lambda x: int(x[len('video'):]))
all_query_results_df

AttributeError: module 'helpers.eval_utils' has no attribute 'twohee_data_col_to_df'

In [68]:
recall_metrics = calculate_recall(all_query_results_df)
map_value = calculate_mean_average_precision(all_query_results_df)

print("Recall metrics at top-1, 5, 10:")
print(recall_metrics)
print("Mean Average Precision (MAP):")
print(map_value)

Recall metrics at top-1, 5, 10:
{'recall@1': 0.426, 'recall@5': 0.716, 'recall@10': 0.814}
Mean Average Precision (MAP):
0.5456543650793645


In [81]:
# Calculate NDCG with k=1, 5, 10
for k in [1, 5, 10]:
    ndcg_value = calculate_ndcg(all_query_results_df, k=k)
    print(f"NDCG@{k}: {ndcg_value}")


NDCG@1: 0.426
NDCG@5: 0.5780321865313451
NDCG@10: 0.6100154270801753


In [83]:
get_all_eval_scores(all_query_results_df)

{'recall@1': 0.426,
 'recall@5': 0.716,
 'recall@10': 0.814,
 'map': 0.5456543650793645,
 'ndcg@1': 0.426,
 'ndcg@5': 0.5780321865313451,
 'ndcg@10': 0.6100154270801753}

## Try evaluation against queries from FIRE benchmark

We are working with a sample of MSR-VTT and our evaluation pipeline supports only one relevant query per video, hence we need to filter the full FIRE benchmark to only include videos we have sampled and ones with a single relevant result.

FIRE_BENCHMARK_Q_JUDGEMENTS is created in the notebook `./clean_fire_judgements.ipynb`

In [140]:
# Run query pipeline using FIRE

def read_frame_search_fire_csv(csv_file):
    with open(csv_file, 'r', encoding='utf-8-sig') as f:
        data = csv.DictReader(f)
        for line in data:
            yield line['video_id'], line['sentence']
            
fire_query_results = DataCollection(video_search_pipeline(FIRE_BENCHMARK_Q_JUDGEMENTS))

In [136]:
get_all_eval_scores(twohee_data_col_to_df(fire_query_results))

{'recall@1': 0.5732484076433121,
 'recall@5': 0.821656050955414,
 'recall@10': 0.9076433121019108,
 'map': 0.6800513092710544,
 'ndcg@1': 0.5732484076433121,
 'ndcg@5': 0.7064313418462581,
 'ndcg@10': 0.7348170561659407}